# Imports

In [101]:
from typing import Any, Optional, Tuple, List
from numpy import float64
from PIL import Image
import graphviz
import pandas as pd
import IPython.display
from IPython.display import display, Image
import folium

# Clases

In [102]:
class Queue:

    def __init__(self, size: int) -> None:
        self.queue: List[Any] = []
        self.size = 1000

    def __repr__(self) -> str:
        return str(self.queue)

    def add(self, elem: Any) -> None:
        if len(self.queue) >= self.size:
            raise ValueError('The Queue is full')
        self.queue.append(elem)

    def remove(self) -> Any:
        if not self.queue:
            raise ValueError('The Queue is empty')
        return self.queue.pop(0)

    def is_empty(self) -> bool:
        return len(self.queue) == 0

In [103]:
class Node:

    def __init__(self, data:'Property' ) -> None:
        self.data = data
        self.left: Optional["Node"] = None
        self.right: Optional["Node"] = None
        self.level = None

    def __repr__(self) -> str:
        left = self.left.data if self.left is not None else None
        right = self.right.data if self.right is not None else None
        return f'{self.data} {left} {right}'

In [104]:
class Arbol_AVL:

    def __init__(self) -> None:
        self.root: Optional["Node"] = None

    def put_levels(self, nodo:'Property') -> bool:
        nivel = 0
        p = self.root
        while p is not None:
            nivel +=1
            if nodo.metric == p.data.metric:
                if nodo.metric2 == p.data.metric2:
                    p.level = nivel
                    return True
                elif nodo.metric2 < p.data.metric2:
                    p = p.left
                else:
                    p = p.right
            else:
                if nodo.metric < p.data.metric:
                    p = p.left
                else:
                    p = p.right
        return False

    def search(self, elem: 'Property') -> Tuple[Optional["Node"], Optional["Node"]]:
        p, pad = self.root, None
        while p is not None:
            if elem.metric == p.data.metric:
                return p, pad
            else:
                pad = p
                if elem.metric < p.data.metric:
                    p = p.left
                else:
                    p = p.right
        return p, pad

    def __search_dupli(self, elem: 'Property',node:'Node'= None) -> Tuple[Optional["Node"], Optional["Node"]]:
        if node is None:
            node = self.root
        nivel = node.level
        p, pad = node, None
        while p is not None:
            pad = p
            if elem.metric == p.data.metric:
                if elem.metric2 > p.data.metric2:
                    p = p.right
                    bol = True
                else:
                    p = p.left
                    bol = False
            else:
                if elem.metric < p.data.metric:
                    p = p.left
                    bol = False
                else:
                    p = p.right
                    bol = True
        return bol, pad

    def search_metric(self, elem: float64) -> Tuple[Optional["Node"], Optional["Node"]]:
        p, aux = self.root, None
        while p is not None:
            if elem == p.data.metric:
                aux = p
                p = None
                #return p
            else:
                pad = p
                if elem < p.data.metric:
                    p = p.left
                else:
                    p = p.right
        self.search_all(aux.data)

    def search_all(self,elem:'Property', mode=1):
        p, pad = self.search(elem)
        if pad is not None:
            pad, abuelo = self.search(pad.data)
            if abuelo is not None:
                if(abuelo.left == pad ):
                    tio = abuelo.right
                else:
                    tio = abuelo.left
                if(mode==1):
                    aux = [p.data,pad.data,tio.data,abuelo.data]
                    map = self.mostrar_geolocalizacion_mapa(aux,2)
                    display(map)
            else:
                tio= None
        else:
            abuelo = None
            tio = None
        factor_b = self.balance_factor(p)
        return p, p.level, factor_b, pad, abuelo, tio

    def insert(self, elem: 'Property') -> bool:
        to_insert = Node(elem)
        if self.root is None:
            self.root = to_insert
            return True
        else:
            bol, pad = self.__search_dupli(elem, self.root)
            if bol:
                pad.right = to_insert
            else:
                pad.left = to_insert
            self.desbalance()
            return False


    def delete(self, elem: Any, mode: bool = True) -> bool:
        if isinstance(elem, Property):
            elem = elem.metric
        if elem is Property:
            elem = elem.metric
        p, pad = self.search(elem)
        if p is not None:
            if p.left is None and p.right is None:
                if p == pad.left:
                    pad.left = None
                else:
                    pad.right = None
                del p
            elif p.left is None and p.right is not None:
                if p == pad.left:
                    pad.left = p.right
                else:
                    pad.right = p.right
                del p
            elif p.left is not None and p.right is None:
                if p == pad.left:
                    pad.left = p.left
                else:
                    pad.right = p.left
                del p
            else:
                if mode:
                    pred, pad_pred = self.__pred(p)
                    p.data.metric = pred.data.metric
                    if pred.left is not None:
                        if pad_pred==p:
                           p.left = pred.left
                        else:
                            pad_pred.right = pred.left
                    else:
                        pad_pred.right = None
                    del pred
                else:
                    sus, pad_sus = self.__sus(p)
                    p.data.metric = sus.data.metric
                    if sus.right is not None:
                        pad_sus.left = sus.right
                    else:
                        pad_sus.left = None
                    del sus
        self.desbalance()

    def __pred(self, node: "Node") -> Tuple["Node", "Node"]:
        p, pad = node.left, node
        while p.right is not None:
            p, pad = p.right, p

        return p, pad

    def __sus(self, node: "Node") -> Tuple["Node", "Node"]:
        p, pad = node.right, node
        while p.left is not None:
            p, pad = p.left, p

        return p, pad

    def preorder(self,node: Optional["Node"]) -> None:
        if node is not None:
            print(node.data, end=' ')
            self.preorder(node.left)
            self.preorder(node.right)

    def __inorder(self, node: Optional["Node"]) -> None:
        if node is not None:
            self.__inorder(node.left)
            print(node)
            self.__inorder(node.right)

    def inorder(self) -> None:
        self.__inorder(self.root)

    def tree_height(self, Node_I) -> int:
        if isinstance(Node, int):
            Node_I = self.search(Node_I)[0]
        if isinstance(Node_I,Property):
            Node_I = self.search(Node_I)[0]
        if Node_I is None:
            return -1
        max_height = -1
        level = 0
        queue = [(Node_I, level)]
        while queue:
            node, current_level = queue.pop(0)
            if node is not None:
                max_height = max(max_height, current_level)
                queue.append((node.left, current_level + 1))
                queue.append((node.right, current_level + 1))
        return max_height

    def balance_factor(self, node: "Node") -> int:
        if isinstance(node,int):
            node = self.search(node)[0]
        if isinstance(node,Property):
            node = self.search(node)[0]
        left_height = self.tree_height(node.left)
        right_height = self.tree_height(node.right)
        return right_height-left_height

    def levels_nr_print(self, node: "Node"= None) -> None:
        if node is None:
            node = self.root
        p, q = node, Queue(50)
        nodos = []
        q.add(p)
        while not q.is_empty():
            p = q.remove()
            nodos.append(p.data)
            print(p.data, p.level , end=' ')
            if p.left is not None:
                q.add(p.left)
            if p.right is not None:
                q.add(p.right)
        #print("cant ",len(nodos))

    def levels_nr(self, node: "Node") -> None:
        p, q = node, Queue(50)
        nodos = []
        q.add(p)
        while not q.is_empty():
            p = q.remove()
            nodos.append(p.data)
            if p.left is not None:
                q.add(p.left)
            if p.right is not None:
                q.add(p.right)
        return nodos

    def desbalance(self) -> 'Node':
        dic, aux = {}, []
        nodos: 'Node'= self.levels_nr(self.root)
        for elem in nodos:
            dic[elem]= self.balance_factor(elem)
            self.put_levels(elem)
            if dic[elem] > 1 or dic[elem] < -1:
                aux.append(elem)
        if len(aux)>0:
            self.balanceo(aux[-1],dic[aux[-1]])

    def balanceo(self, node, equi):
        if isinstance(node, int):
            node = self.search(node)[0]
        if isinstance(node, Property):
            node = self.search(node)[0]

        if equi>1:
            if self.balance_factor(node.right)>=0:
                node = self.rotacion_Sim_Iz(node)
            else:
                aux= node
                node = self.rotacion_Sim_De(node.right)
                node = self.rotacion_Sim_Iz(aux)

        else:
            if self.balance_factor(node.left)<=0:
                node = self.rotacion_Sim_De(node)
            else:
                aux = node
                node = self.rotacion_Sim_Iz(node.left)
                node = self.rotacion_Sim_De(aux)


  #si es 2 rotacion a la izquierda, -2 rotacion a la derecha
  # signos iguales rotacion simple, signos contrarios rotacion doble

    def rotacion_Sim_Iz(self,nodo:'Node'):
        if isinstance(nodo,int):
            nodo = self.search(nodo)[0]
        if isinstance(nodo,Property):
            nodo = self.search(nodo)[0]
        padre = self.search(nodo.data)[1]
        aux = nodo.right
        nodo.right = aux.left
        aux.left = nodo
        if self.root.data == nodo.data:
            self.root = aux
        else:
            if padre.data.metric > nodo.data.metric:
                padre.left= aux
            else:
                padre.right = aux
        return aux

    def rotacion_Sim_De(self,nodo:'Node'):
        if isinstance(nodo,int):
            nodo = self.search(nodo)[0]
        if isinstance(nodo,Property):
            nodo = self.search(nodo)[0]
        padre = self.search(nodo.data)[1]
        aux= nodo.left
        nodo.left= aux.right
        aux.right=nodo
        if self.root.data == nodo.data:
            self.root = aux
        else:
            if padre.data.metric > nodo.data.metric:
                padre.left= aux
            else:
                padre.right = aux
        return aux

    def integer_to_dms(self, integer_value):
        # Degrees
        degrees = int(integer_value)

        # Calculate the fractional part
        fractional_part = abs(integer_value - degrees)

        # Minutes
        minutes = int(fractional_part * 60)

        # Seconds
        seconds = (fractional_part * 60 - minutes) * 60
        degrees = f"{degrees}° {minutes}' {seconds:.2f}\""
        return degrees

    def graficar_arbol(self,nodo= None):
        if (nodo is None):
            nodo = self.root
        grafo = self.__graficar_arbol(nodo)
        grafo.render('arbol_Properties', view=True)
        return grafo

    def __graficar_arbol(self, nodo:"Node", grafo=None):
        if grafo is None:
            grafo = graphviz.Digraph(format='pdf')
        if nodo is not None:
            etiqueta = f"—{nodo.data.metric2}—\n{str(nodo.data.city)}\nLatitud: {self.integer_to_dms(nodo.data.latitude)}\nLongitud: {self.integer_to_dms(nodo.data.longitude)}\n"
            etiqueta+= f"Precio: ${str(int(nodo.data.price))}\nSuperficie Total: {str(int(nodo.data.surface_total))}m²\n Métrica: {str(int(nodo.data.metric))} \nFactor de Balanceo: {self.balance_factor(nodo)}"
            grafo.node(str(nodo.data.metric2), label= etiqueta, color='antiquewhite4', fontcolor='gray15', style='filled', fillcolor='beige', penwidth='3')
            if nodo.left is not None:
                grafo.edge(str(nodo.data.metric2), str(nodo.left.data.metric2), color='aquamarine3', penwidth='2')
                self.__graficar_arbol(nodo.left, grafo)
            if nodo.right is not None:
                grafo.edge(str(nodo.data.metric2), str(nodo.right.data.metric2), color='mediumslateblue', penwidth='2')
                self.__graficar_arbol(nodo.right, grafo)
        return grafo

    def niveles_recursivo(self):
        resultado = []
        arbol = self.root
        self.recorrer_niveles([arbol], resultado)
        return resultado

    def recorrer_niveles(self, nodos, resultado):
        if not nodos:
            return
        siguiente_nivel = []
        nivel_actual = []
        for nodo in nodos:
            nivel_actual.append(nodo.data.metric2)
            if nodo.left:
                siguiente_nivel.append(nodo.left)
            if nodo.right:
                siguiente_nivel.append(nodo.right)
        resultado.extend(nivel_actual)
        self.recorrer_niveles(siguiente_nivel, resultado)

    def __buscar_criterios(self, filtro, condi: list, nodo: list['Property']):
        attribute_mapping = {
            1: 'title',
            2: 'department',
            3: 'city',
            4: 'property_type',
            5: 'latitude',
            6: 'longitude',
            7: 'surface_total',
            8: 'surface_covered',
            9: 'bedrooms',
            10: 'bathrooms',
            11: 'operation_type',
            12: 'price'}
        aux = []
        if filtro in attribute_mapping:
            attribute = attribute_mapping[filtro]
            operator = condi[1]
            value = condi[0]
            for elem in nodo:
                attr_value = getattr(elem, attribute)
                if operator == "=" and attr_value == value:
                    aux.append(elem)
                elif operator == "!=" and attr_value != value:
                    aux.append(elem)
                elif operator == ">" and attr_value > value:
                    aux.append(elem)
                elif operator == "<" and attr_value < value:
                    aux.append(elem)
                elif operator == ">=" and attr_value >= value:
                    aux.append(elem)
                elif operator == "<=" and attr_value <= value:
                    aux.append(elem)
        return aux

    def buscar_criterios(self, condi1:list, filtro1, condi2:list=None, filtro2=None, condi3:list=None, filtro3=None):
        auxi = []
        if condi1 is not None:
            auxi = self.__buscar_criterios(filtro1, condi1, self.levels_nr(self.root))
        if condi2 is not None:
           auxi = self.__buscar_criterios(filtro2, condi2, auxi)
        if condi3 is not None:
            auxi = self.__buscar_criterios(filtro3, condi3, auxi)
        padre, tio, abuelo, crite = [], [], [], []
        for i, elem in enumerate(auxi):
            elem.familia += "Original\t"
            crite.append(self.search_all(elem, 2)[0])
            padre.append(self.search_all(elem, 2)[3])
            if padre[i] is not None:
                padre[i].data.familia += f"Padre de {elem.metric2}.\t"
            tio.append(self.search_all(elem, 2)[4])
            if tio[i] is not None:
                tio[i].data.familia += f"Tio de {elem.metric2}.\t"
            abuelo.append(self.search_all(elem, 2)[5])
            if abuelo[i] is not None:
                abuelo[i].data.familia += f"Abuelo de {elem.metric2}.\t"
        repet = self.quitar_repeticiones([crite, padre, tio, abuelo])
        map = self.mostrar_geolocalizacion_mapa([auxi, repet[1], repet[2], repet[3]], 3)
        display(map)
        return auxi

    def quitar_repeticiones(self, lista:list):
        for i in range(len(lista)-1):
            for j in range(len(lista[i])):
                if lista[i+1][j] in lista[0]:
                    lista[i+1][j] = None
        return lista

    def mostrar_geolocalizacion_mapa(self,nodos:list["Node"],mode=1):
        if mode==1:
            mapa = folium.Map(location=[nodos[0].latitude, nodos[0].longitude], zoom_start=13)
            for elem in nodos:
                etiqueta = f"""{elem.metric2}\n{str(elem.city)}\nLatitud: {arbol.integer_to_dms(elem.latitude)}\n
                Longitud: {str(elem.longitude)}\nPrecio: ${str(int(elem.price))}\nSuperficie Total: {str(int(elem.surface_total))}m²\n"""
                folium.Marker([elem.latitude, elem.longitude], tooltip= etiqueta, popup= etiqueta).add_to(mapa)
        elif mode ==2:
            mapa = folium.Map(location=[nodos[0].latitude, nodos[0].longitude], zoom_start=13)
            nombre = ["Original", "Padre", "Tio", "Abuelo"]
            for i, elem in enumerate(nodos):
                etiqueta = f"""{nombre[i]}\n{elem.metric2}\n{str(elem.city)}\nLatitud: {arbol.integer_to_dms(elem.latitude)}\n
                Longitud: {str(elem.longitude)}\nPrecio: ${str(int(elem.price))}\nSuperficie Total: {str(int(elem.surface_total))}m²\n"""
                folium.Marker([elem.latitude, elem.longitude], tooltip= nombre[i], popup= etiqueta).add_to(mapa)
        else:
            mapa = folium.Map(location=[nodos[0][0].latitude, nodos[0][0].longitude], zoom_start=13)
            nombre = ["Original", "Padre", "Tio", "Abuelo"]
            colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'pink', 'cadetblue'] * 12
            for i in range(len(nodos[0])):
                etiqueta = f"""{nodos[0][i].familia}{nodos[0][i].metric2}\n{str(nodos[0][i].city)}\nLatitud: {arbol.integer_to_dms(nodos[0][i].latitude)}\n
                Longitud: {str(nodos[0][i].longitude)}\nPrecio: ${str(int(nodos[0][i].price))}\nSuperficie Total: {str(int(nodos[0][i].surface_total))}m²\n"""
                folium.Marker([nodos[0][i].latitude, nodos[0][i].longitude], tooltip= f"{nodos[0][i].familia}", popup= etiqueta, min_width=900, icon=folium.Icon(color=colores[i]),icon_size=(500, 500)).add_to(mapa)
                if nodos[1][i] is not None:
                    etiqueta = f"""{nodos[1][i].data.familia}{nodos[1][i].data.metric2}\n{str(nodos[1][i].data.city)}\nLatitud: {arbol.integer_to_dms(nodos[1][i].data.latitude)}\n
                    Longitud: {str(nodos[1][i].data.longitude)}\nPrecio: ${str(int(nodos[1][i].data.price))}\nSuperficie Total: {str(int(nodos[1][i].data.surface_total))}m²\n"""
                    folium.Marker([nodos[1][i].data.latitude, nodos[1][i].data.longitude],tooltip= f"{nodos[1][i].data.familia}", popup= etiqueta, min_width=900, icon=folium.Icon(color=colores[i]), icon_size=(200, 200)).add_to(mapa)
                    if nodos[3][i] is not None:
                        etiqueta = f"""{nodos[3][i].data.familia}{nodos[3][i].data.metric2}\n{str(nodos[3][i].data.city)}\nLatitud: {arbol.integer_to_dms(nodos[3][i].data.latitude)}\n
                        Longitud: {str(nodos[3][i].data.longitude)}\nPrecio: ${str(int(nodos[3][i].data.price))}\nSuperficie Total: {str(int(nodos[3][i].data.surface_total))}m²\n"""
                        folium.Marker([nodos[3][i].data.latitude, nodos[3][i].data.longitude],tooltip= f"{nodos[3][i].data.familia}", popup= etiqueta, min_width=900, icon=folium.Icon(color=colores[i]), icon_size=(100, 100)).add_to(mapa)
                        if(nodos[2][i] is not None):
                            etiqueta = f"""{nodos[2][i].data.familia}{nodos[2][i].data.metric2}\n{str(nodos[2][i].data.city)}\nLatitud: {arbol.integer_to_dms(nodos[2][i].data.latitude)}\n
                            Longitud: {str(nodos[2][i].data.longitude)}\nPrecio: ${str(int(nodos[2][i].data.price))}\nSuperficie Total: {str(int(nodos[2][i].data.surface_total))}m²\n"""
                            folium.Marker([nodos[2][i].data.latitude, nodos[2][i].data.longitude],tooltip= f"{nodos[2][i].data.familia}", popup= etiqueta, min_width=900, icon=folium.Icon(color=colores[i]), icon_size=(100, 100)).add_to(mapa)
        mapa.save('mapa2.html')
        return mapa

In [105]:
class Property:

    def __init__(self, title:'str', department:'str', city:'str', property_type:'str', latitude:'float', longitude:'float', surface_total:'float',
                 surface_covered:'float', bedrooms:'int', bathrooms:'int', operation_type:'str', price: 'float') -> None:
        self.title = title
        self.department = department
        self.city = city
        self.property_type = property_type
        self.latitude = latitude
        self.longitude = longitude
        self.surface_total = surface_total
        self.surface_covered = surface_covered
        self.bedrooms = bedrooms
        self.bathrooms = bathrooms
        self.operation_type = operation_type
        self.price = price
        self.metric = self.price/ self.surface_total
        self.metric2 = self.title.split("Cod. ")[1]
        self.familia = ""

    def __str__(self) -> str:
        return f"{self.title.split('Cod. ')[1]}"
        return f"{self.title} {self.department} {self.city} {self.property_type} {self.latitude} {self.longitude} {self.surface_total} {self.surface_covered} {self.bedrooms} {self.bathrooms} {self.operation_type} {self.price} "


# Pruebas

In [106]:
# Dataframe
df = pd.read_csv('sample_data/co_properties_final.csv')
df['metric'] = df['price'].divide(df['surface_total'])

# Tree
arbol = Arbol_AVL()
aux= []
for index, row in df.iterrows():
    aux.append(Property(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7],row[8], row[9], row[10], row[11]))
    arbol.insert(aux[index])

# Shows the map
busqueda = arbol.buscar_criterios([100,">"],7,[1730000,">="],12)

In [107]:
# Show the Graph with given criteria
arbol_criterios = Arbol_AVL()
criterios = arbol.buscar_criterios([4,">"], 9, [15000, ">="], 12, [200,">"], 7)
for data in criterios:
    arbol_criterios.insert(Property(data.title, data.department, data.city, data.property_type, data.latitude, data.longitude, data.surface_total, data.surface_covered, data.bedrooms, data.bathrooms, data.operation_type, data.price))
image = Image(data=arbol_criterios.graficar_arbol().pipe(format='pdf'))
display(image)

In [108]:
#Print the column names
columns = df.columns.tolist()
for i, colum in enumerate(columns):
    print(f'{i+1}.  {colum.capitalize().replace("_", " ")}' if i < 9 else f'{i+1}. {colum.capitalize().replace("_", " ")}')

1.  Title
2.  Department
3.  City
4.  Property type
5.  Latitude
6.  Longitude
7.  Surface total
8.  Surface covered
9.  Bedrooms
10. Bathrooms
11. Operation type
12. Price
13. Metric


In [109]:
# Show the Full Graph
# image = Image(data=arbol.graficar_arbol().pipe(format='pdf'))
# display(image)